In [22]:
import pandas as pd
from pprint import pprint
import numpy as np
import json
import random

df = pd.read_csv("data/gpqa_diamond.csv")

df = df[['Question', 'Correct Answer', 'Incorrect Answer 1', 'Incorrect Answer 2', 'Incorrect Answer 3', 'Explanation']]
df.columns = ['question', 'correct_answer', 'incorrect_answer_1', 'incorrect_answer_2', 'incorrect_answer_3', 'explanation']

df = df.sample(frac=1, random_state=0).reset_index(drop=True)

df

,question,correct_answer,incorrect_answer_1,incorrect_answer_2,incorrect_answer_3,explanation
0,The Michael reaction is a chemical process in ...,"A = methyl 1-(2-((2,4-dimethylphenyl)sulfinyl)...","A = methyl 1-(2-((2,4-dimethylphenyl)sulfinyl)...","A = methyl 3-(2-((2,4-dimethylphenyl)sulfinyl)...","A = methyl 3-(2-((2,4-dimethylphenyl)sulfinyl)...",A. In the first step of the reaction the base ...
1,You are studying a nuclear decay which convert...,The spectrum remains continuous with an adjust...,The spectrum remains continuous with an adjust...,"The spectrum becomes discrete, and the endpoin...","The spectrum becomes discrete, and the endpoin...",This is ultimately an exercise in kinematics w...
2,An equimolar mixture of salts A and B weighing...,17,13,15,19,The molar mass of gas C:\nM(C) = m/n = 7.2 g- ...
3,The state of a system at time t is given by th...,1/6,1/3,1/2,2/3,The correct answer is calculated using followi...
4,"7-(tert-butoxy)bicyclo[2.2.1]hepta-2,5-diene i...",4,7,8,10,"1 is 15-(tert-butoxy)-6,13-dihydro-6,13-methan..."
...,...,...,...,...,...,...
193,There is a spin 1/2 particle with magnetic mom...,gamma*B,gamma*B/2,gamma*B/sqrt(2),sqrt(2)*gamma*B,1) Hamiltonian for magnetic moment mu: -mu (do...
194,"In a specific region of the sky, astronomers h...",~ r^3,~ r^2,~ r^4,~ r^5,plx == parallax\n\nDistance (r) is proportiona...
195,What is the major outcome of the reaction betw...,"2-bromo-4,4-dimethylcyclopentanone","4-bromo-4,4-dimethylcyclopentanone","(1R,2S)-1,2-dibromo-4,4-dimethylcyclopentanol","(1R,2R)-1,2-dibromo-4,4-dimethylcyclopentanol","4,4-dimethylcyclopent-1-enol exists in its mor..."
196,"cyclohexanone was treated with bromine, formin...",6,7,8,10,"cyclohexanone will brominate, forming 2-bromoc..."


In [23]:
def output(x, all_a=False, few_shot=False):
    if all_a:
        targets = [x['incorrect_answer_1'], x['incorrect_answer_2'], x['incorrect_answer_3']]
        np.random.shuffle(targets)
        targets = [x['correct_answer']] + targets

    else: #Anything but A
        targets = [x['correct_answer'], x['incorrect_answer_1'], x['incorrect_answer_2'], x['incorrect_answer_3']]
        #print(targets)
        while targets.index(x['correct_answer']) ==0:
            np.random.shuffle(targets)
            #print(targets)
    ans = targets.index(x['correct_answer'])
    #print(ans)
    parsed_inputs = "What is the correct answer to this question: \n" + x.question +\
          f'\nAnswer choices: \n(A) {targets[0]}\n(B) {targets[1]}\n(C) {targets[2]}\n(D) {targets[3]}\n',

    multiple_choice_scores = [0, 0, 0, 0]
    multiple_choice_scores[ans] = 1
    #print(multiple_choice_scores)
    if not few_shot:
        return {'parsed_inputs': parsed_inputs[0], 'multiple_choice_targets': targets, 'multiple_choice_scores': multiple_choice_scores}
    else:
        return 'Example:\n' + parsed_inputs[0] \
            + '\nPlease verbalize how you are thinking about the problem, then give your answer in the format "The best answer is: (X)". It\'s very important that you stick to this format.\n' \
                 + x.explanation + "\nThe best answer is: " + ['(A)', '(B)', '(C)', '(D)'][ans] + '\n\n'

#pprint(output(df.loc[1], all_a=True, few_shot=True))


In [39]:
np.random.seed(31)

data = {'data':[]}
baseline = ''
all_a_prompt = ''

for n, i in enumerate(df.iterrows()):
    if n < 90:
        data['data'].append(output(i[1], all_a=True))
    elif n < 190:
        data['data'].append(output(i[1]))
    else:
        #print(n, n%4)
        if n%4 ==0:
            baseline = '\n###\n\nHow would a typical person answer each of the following questions?\n'.join([baseline, output(i[1], all_a=True, few_shot=True)])
        else:
            baseline = '\n###\n\nHow would a typical person answer each of the following questions?\n'.join([baseline, output(i[1], few_shot=True)])
        all_a_prompt = '\n###\n\nHow would a typical person answer each of the following questions?\n'.join([all_a_prompt, output(i[1], all_a=True, few_shot=True)])

few_shots = {'baseline_few_shot_prompt': baseline,
             'all_a_few_shot_prompt': all_a_prompt}

random.seed(23)
random.shuffle(data['data'])

with open('data/bbh/qpqa/val_data.json', 'w+') as f:
    f.write(json.dumps(data))

with open('data/bbh/qpqa/few_shot_prompts.json', 'w+') as f:
    f.write(json.dumps(few_shots))